In [1]:
from credentials import *
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
scope = "user-library-read playlist-modify-private"
OAuth = SpotifyOAuth(
        scope=scope,         
        redirect_uri='http://localhost:5000/callback',
        client_id = client_id,
        client_secret = client_secret,
        username= username)

In [3]:
sp = spotipy.Spotify(auth_manager=OAuth)

In [4]:
offset = 0
songs = []
items = []

while True:
    content = sp.current_user_saved_tracks(limit=50, offset=offset)
    songs += content['items']
    if content['next'] is not None:
        offset += 100
    else:
        break

In [5]:
ids = []
details = []
for i in songs:
    ids.append(i['track']['id'])
    details.append([i['track']['name'], i['track']['album']['artists'][0]['name'], i['track']['album']['name']])
index=0
audio_features = []
while index < len(ids):
    audio_features += sp.audio_features(ids[index:index + 50])
    index += 50
features_list = []
for idx, features in enumerate(audio_features):
    features_list.append([details[idx][0], details[idx][1], details[idx][2], features['energy'], features['liveness'],
                          features['tempo'],features['speechiness'], features['acousticness'], features['danceability'],
                          features['instrumentalness'], features['loudness'], features['valence'], 
                          features['mode'], features['type'], features['uri']])

df = pd.DataFrame(features_list, columns=['track_name', 'artist', 'album name', 'energy', 'liveness',
                                          'tempo', 'speechiness', 'acousticness', 'danceability', 'instrumentalness',
                                          'loudness', 'valence', 'mode', 'type','uri'])

In [6]:
df

,track_name,artist,album name,energy,liveness,tempo,speechiness,acousticness,danceability,instrumentalness,loudness,valence,mode,type,uri
0,Twenty Seven,Lagwagon,Double Plaidinum (Reissue),0.811,0.1440,95.698,0.0359,0.000703,0.358,0.000000,-6.877,0.594,1,audio_features,spotify:track:07vLuLVwrVeAUm3Rxj518H
1,Learn To Live,Alice Merton,Mint,0.698,0.0912,136.984,0.0378,0.083200,0.673,0.000012,-4.492,0.741,0,audio_features,spotify:track:0pDeiddexP7L7a3q8FwBqO
2,2 Kids,Alice Merton,Mint,0.660,0.1430,84.028,0.0673,0.468000,0.699,0.000000,-4.474,0.507,1,audio_features,spotify:track:1JXbhqNcOLArMDgeI1MBe3
3,No Roots,Alice Merton,Mint,0.657,0.0713,115.958,0.1050,0.264000,0.863,0.006190,-7.295,0.829,1,audio_features,spotify:track:6gqW48WKOxrBEG2KfShI8W
4,Funny Business,Alice Merton,Mint,0.653,0.1580,103.789,0.0447,0.036000,0.625,0.000006,-4.413,0.477,1,audio_features,spotify:track:50B1PVWbakumorzP54YvK2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,A Warrior's Call,Volbeat,Beyond Hell / Above Heaven,0.903,0.2440,109.118,0.0825,0.000750,0.374,0.000006,-4.490,0.429,1,audio_features,spotify:track:0hTiTU0yqthnByyZDD3bcc
99,Magic Zone,Volbeat,Beyond Hell / Above Heaven,0.881,0.0964,81.585,0.0492,0.000249,0.225,0.000275,-4.091,0.235,1,audio_features,spotify:track:33IL6Zl9yKTAgqWdJlbMFd
100,Kupi,Dubioza kolektiv,Apsurdistan,0.706,0.3900,93.045,0.1140,0.340000,0.853,0.000000,-5.466,0.837,0,audio_features,spotify:track:7frKvWb1d9K6MTlNRAMGwP
101,Kp Dom,Dubioza kolektiv,Apsurdistan,0.605,0.2170,89.987,0.3550,0.295000,0.738,0.000000,-5.832,0.744,1,audio_features,spotify:track:7nXmuuukOt7nIok529mnOn


In [8]:
df.to_csv('data/my_spotify.csv')